1. Load batch data
2. Loop cases
3. Push results back to staging SQLite

In [ ]:
import sqlite3
import pandas as pd
import json

In [ ]:
df = pd.read_parquet("data/formation_sample.parquet.gzip")
n = len(df)

In [ ]:
run_name = "olvi"
conn = sqlite3.connect("data/eval.db")
cursor = conn.cursor()
y = cursor.execute(f"SELECT DISTINCT id FROM {run_name}")
record = y.fetchall()
len(record)

In [ ]:
ids = [r[0] for r in record]
print(ids)

In [ ]:
y = cursor.execute(f"SELECT * FROM {run_name} LIMIT 10")
record = y.fetchall()

In [ ]:
def get_output_info(output: str, route: list[str]) -> str:
    """Get the information from the output."""
    response = json.loads(output)
    for r in route:
        response = response[r]
    return response


def parse_output(output: str) -> list[dict]:
    """Parse the json output from the database."""

    response = json.loads(output)
    return json.loads(response["message"]["content"])

In [ ]:
get_output_info(record[0][2], route=["eval_duration"])

In [ ]:
eval_durations = []
for r in record:
    eval_durations.append(get_output_info(r[2], route=["eval_duration"]))

expected_finish_hours = n * (max(eval_durations) / 1e9) / 3600 / 6

import datetime

finish_time = datetime.datetime.now() + datetime.timedelta(hours=expected_finish_hours)
print(f"ETA: {finish_time.strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
parse_output(record[0][2])